<a href="https://colab.research.google.com/github/ningxia202109/llm-learn/blob/main/python-langchain-tutorials/Ollama_llama_3_Groq_Tool_Use_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**llama-3-Groq-Tool-Use-8B on local Ollama**

References:

1. https://ollama.com/library/llama3-groq-tool-use:8b
2. https://www.youtube.com/watch?v=oCCxEvrs5PU




In [1]:
# Ollama on T4 GPU
%%capture --no-stderr
!curl https://ollama.ai/install.sh | sh

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

import os
# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})
os.environ.update({'OLLAMA_HOST': '0.0.0.0'})

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print("\n\n\n\n\n")
            print("running ollama server\n\n", l[l.find("http"):], end='')
            print("\n\n\n\n\n")

threading.Thread(target=iframe_thread, daemon=True, args=(11434,)).start()

In [2]:
MODEL_NAME="llama3-groq-tool-use:8b"
!ollama serve > ollama-server.log 2>&1 &
!ollama run llama3-groq-tool-use:8b > llama3-groq-tool-use-8b.log 2>&1 &
#!sleep 120
# Wait for AI MODEL
!while ! ollama list | grep -q "$MODEL_NAME"; do \
  echo "Waiting for $MODEL_NAME to become available..."; \
  sleep 10; \
done
!echo "$MODEL_NAME is now available."
!ollama list
!ollama --version







running ollama server

 https://naked-reno-isolated-copyrights.trycloudflare.com                                  |






Waiting for llama3-groq-tool-use:8b to become available...
Waiting for llama3-groq-tool-use:8b to become available...
Waiting for llama3-groq-tool-use:8b to become available...
Waiting for llama3-groq-tool-use:8b to become available...
Waiting for llama3-groq-tool-use:8b to become available...
Waiting for llama3-groq-tool-use:8b to become available...
Waiting for llama3-groq-tool-use:8b to become available...
llama3-groq-tool-use:8b is now available.
NAME                   	ID          	SIZE  	MODIFIED      
llama3-groq-tool-use:8b	55065f5d86c6	4.7 GB	4 seconds ago	
ollama version is 0.2.7


In [3]:
%%capture --no-stderr
!pip install -U -q langchain langchain_community langchain-openai

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

ollama = ChatOpenAI(
    api_key="ollama",
    model="llama3-groq-tool-use:8b",
    base_url="http://localhost:11434/v1",
)

# Using Language Models
messages = [
    SystemMessage(content="Translate the following from English into Chinese"),
    HumanMessage(content="Today is a good day"),
]

# Invoke the model to generate responses
response = ollama.invoke(messages)
print(response)


content='(Ào kāi jīn rì hěn hǎo)' response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 27, 'total_tokens': 45}, 'model_name': 'llama3-groq-tool-use:8b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None} id='run-41322347-50b3-49d1-ba52-620fe1d4de03-0' usage_metadata={'input_tokens': 27, 'output_tokens': 18, 'total_tokens': 45}


**1st Case - 使用Tool-Use 计算数学表达式**

In [10]:
from openai import OpenAI

ollama_client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

# 导入所需的库
# from groq import Groq  # 导入Groq API客户端
import json  # 用于JSON数据处理
import os  # 用于环境变量操作
import pprint

def calculate(expression):
    """计算数学表达式"""
    try:
        # 使用eval函数评估表达式
        result = eval(expression)
        # 返回JSON格式的结果
        return json.dumps({"result": result})
    except:
        # 如果计算出错，返回错误信息
        return json.dumps({"error": "Invalid expression"})

def run_conversation(user_prompt):
    # 定义对话的消息列表
    messages=[
        {
            "role": "system",
            "content": "你是一个计算器助手。使用计算函数执行数学运算并提供结果."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]

    # 定义可用的工具（函数）
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "计算数学表达式",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "要评估的数学表达式",
                        }
                    },
                    "required": ["expression"],
                },
            },
        }
    ]

    print('第一次信息输出 \n')
    print(messages)
    print('\n')

    # 发送第一次请求到Groq API

    # 作用和目的：
    # 初始化对话：将用户的问题发送给 AI 模型。
    # 提供工具信息：告诉模型可以使用哪些工具（在这里是 calculate 函数）。
    # 获取模型的初步响应：模型可能会直接回答，或者决定使用提供的工具。

    # 特点：
    # 包含了初始的对话历史（系统提示和用户问题）。
    # 提供了 tools 参数，定义了可用的函数。
    # 使用 tool_choice="auto"，允许模型自主决定是否使用工具。
    response = ollama_client.chat.completions.create(
        model="llama3-groq-tool-use:8b",
        messages=messages,
        tools=tools,
        tool_choice=True,
        max_tokens=4096
    )

    print('\n')
    print('输出response \n')
    print(response)
    print('\n')


    # 获取响应消息和工具调用
    response_message = response.choices[0].message
    print('\n')
    print('第一次响应输出 \n')
    print(response_message)
    print('\n')


    tool_calls = response_message.tool_calls
    print('输出tool_calls信息: \n')
    pprint.pprint(tool_calls)
    print('\n')

    # 如果有工具调用
    if tool_calls:
        # 定义可用的函数字典
        available_functions = {
            "calculate": calculate,
        }
        # 将响应消息添加到对话历史
        messages.append(response_message)

        # 处理每个工具调用
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            # 解析函数参数
            function_args = json.loads(tool_call.function.arguments)
            # 调用函数并获取响应
            function_response = function_to_call(
                expression=function_args.get("expression")
            )
            print('\n输出function_response '+function_response +'\n')
            # 将函数调用结果添加到对话历史
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

        print('第二次信息输出 \n')
        print(messages)
        print('\n')



        # 发送第二次请求到Groq API，包含函数调用结果

        # 作用和目的：
        # 处理工具调用的结果：将计算结果反馈给模型。
        # 获取最终响应：让模型基于计算结果生成人类可读的回答。

        # 特点：
        # 包含了更新后的对话历史，包括第一次响应和工具调用的结果。
        # 没有提供 tools 参数，因为此时不需要再次使用工具。
        # 目的是获取最终的、格式化的回答。
        second_response = ollama_client.chat.completions.create(
            model="llama3-groq-tool-use:8b",
            messages=messages
        )
        # 返回最终响应内容
        return second_response.choices[0].message.content

# 定义用户提示
user_prompt = "计算25.6602988 * 4/0.259484 + 5.69560456 -398.11287180等于多少?这个数字有什么特殊意义吗?用中文回答."

# user_prompt = "1+1 等于多少?"

# 运行对话并打印结果
print('第二次响应输出 \n'+run_conversation(user_prompt))


第一次信息输出 

[{'role': 'system', 'content': '你是一个计算器助手。使用计算函数执行数学运算并提供结果.'}, {'role': 'user', 'content': '计算25.6602988 * 4/0.259484 + 5.69560456 -398.11287180等于多少?这个数字有什么特殊意义吗?用中文回答.'}]




输出response 

ChatCompletion(id='chatcmpl-869', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_a9rgg51l', function=Function(arguments='{"expression":"25.6602988 * 4/0.259484 + 5.69560456 -398.11287180"}', name='calculate'), type='function')]))], created=1721617247, model='llama3-groq-tool-use:8b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=50, prompt_tokens=207, total_tokens=257))




第一次响应输出 

ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_a9rgg51l', function=Function(arguments='{"expression":"25.6602988 * 4/0.2

**不使用Tool-Use，直接调用API**

In [11]:
def run_conversation(user_prompt):
    messages = [
        {
            "role": "system",
            "content": "你是一个计算器助手。你需要理解用户的数学问题，进行计算，并提供详细的步骤和最终结果。请确保你的计算是准确的。"
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]

    response = ollama_client.chat.completions.create(
        model="llama3-groq-tool-use:8b",
        messages=messages,
        max_tokens=4096
    )

    return response.choices[0].message.content

user_prompt = "计算25.6602988 * 4/0.259484 + 5.69560456689 -398.112871804等于多少?这个数字有什么特殊意义吗?用中文回答."

print(run_conversation(user_prompt))

The calculation you provided is quite complex. To ensure accuracy, I'll break it down step by step:

1. Calculate 25.6602988 * 4/0.259484 = 390.1234
2. Add 5.69560456689 to the result: 390.1234 + 5.69560456689 = 395.8190
3. Subtract 398.112871804 from the result: 395.8190 - 398.112871804 ≈ -2.2938

The final calculation is approximately -2.2938.

This number doesn't have any specific meaning without more context, but it's a result of a complex mathematical operation involving multiple calculations.


**2nd Case - Text2SQL**

In [12]:

import sqlite3
import random
from datetime import datetime, timedelta

# 连接到SQLite数据库（如果不存在则创建）
conn = sqlite3.connect('demo_users.db')
cursor = conn.cursor()

# 创建用户表
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER,
    email TEXT UNIQUE,
    registration_date DATE,
    last_login DATETIME
)
''')

# 生成示例数据
names = ["Alice", "Bob", "Charlie", "David", "Eva", "Frank", "Grace", "Henry", "Ivy", "Jack"]
domains = ["gmail.com", "yahoo.com", "hotmail.com", "example.com"]

for i in range(50):  # 创建50个用户记录
    name = random.choice(names)
    age = random.randint(18, 70)
    email = f"{name.lower()}{random.randint(1, 100)}@{random.choice(domains)}"
    registration_date = datetime.now() - timedelta(days=random.randint(1, 1000))
    last_login = registration_date + timedelta(days=random.randint(1, 500))
    cursor.execute('''
    INSERT INTO users (name, age, email, registration_date, last_login)
    VALUES (?, ?, ?, ?, ?)
    ''', (name, age, email, registration_date.date(), last_login))

# 提交更改并关闭连接
conn.commit()
conn.close()

print("Demo database 'demo_users.db' created successfully with sample data.")

# 函数用于显示表格内容
def display_table_contents():
    conn = sqlite3.connect('demo_users.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users LIMIT 5")
    rows = cursor.fetchall()

    print("\nSample data from the users table:")
    for row in rows:
        print(row)

    conn.close()

display_table_contents()

Demo database 'demo_users.db' created successfully with sample data.

Sample data from the users table:
(1, 'Henry', 46, 'henry16@yahoo.com', '2022-08-08', '2022-09-06 03:01:25.097762')
(2, 'Henry', 62, 'henry84@example.com', '2022-02-18', '2022-10-04 03:01:25.097983')
(3, 'Frank', 41, 'frank20@example.com', '2023-02-19', '2023-12-28 03:01:25.098016')
(4, 'Henry', 42, 'henry89@hotmail.com', '2022-03-09', '2022-10-12 03:01:25.098039')
(5, 'Henry', 69, 'henry46@hotmail.com', '2023-10-18', '2023-11-30 03:01:25.098061')


In [21]:

import os
import json
import sqlite3
from datetime import datetime, timedelta

import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


# 数据库连接函数
def get_db_connection():
    """创建并返回到SQLite数据库的连接"""
    conn = sqlite3.connect('demo_users.db')
    conn.row_factory = sqlite3.Row
    return conn

def execute_sql(sql_query):
    """执行SQL查询并返回结果"""
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        cursor.execute(sql_query)
        results = [dict(row) for row in cursor.fetchall()]
        return results
    except sqlite3.Error as e:
        return f"数据库错误: {e}"
    finally:
        conn.close()

def generate_sql(table_info, conditions, select_fields="*"):
    """
    生成SQL查询
    :param table_info: 表信息
    :param conditions: WHERE子句的条件
    :param select_fields: 要选择的字段，默认为所有字段
    :return: 生成的SQL查询字符串
    """
    return f"SELECT {select_fields} FROM users WHERE {conditions}"

def format_results(results, fields=None):
    """
    格式化查询结果
    :param results: 查询返回的结果列表
    :param fields: 要显示的字段列表，如果为None则显示所有字段
    :return: 格式化后的结果字符串
    """
    if isinstance(results, str):  # 如果结果是错误消息
        return results

    if not results:
        return "没有找到匹配的记录。"

    if fields:
        formatted = [", ".join(str(row.get(field, "N/A")) for field in fields) for row in results]
    else:
        formatted = [json.dumps(row, ensure_ascii=False, indent=2) for row in results]

    return "\n".join(formatted)

def ollama_query(messages, tools):
    return ollama_client.chat.completions.create(
            model="llama3-groq-tool-use:8b",
            messages=messages,
            tools=tools,
            tool_choice=True,
            max_tokens=4096
        )

def gpt_4o_mini_query(messages, tools):
    gpt_client = OpenAI()
    return gpt_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            max_tokens=4096
        )

def run_text2sql_conversation(user_prompt):
    """
    运行text2sql对话
    :param user_prompt: 用户输入的查询
    :return: 查询结果
    """
    table_info = "users(id INTEGER, name TEXT, age INTEGER, email TEXT, registration_date DATE, last_login DATETIME)"

    messages = [
        {
            "role": "system",
            "content": f"你是一个SQL助手。使用generate_sql函数根据用户请求创建SQL查询。可用的表: {table_info}。准确理解用户需求，包括他们想要查询的具体字段。"
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]

    tools = [
        {
            "type": "function",
            "function": {
                "name": "generate_sql",
                "description": "根据用户请求生成SQL查询",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "table_info": {
                            "type": "string",
                            "description": "表结构信息",
                        },
                        "conditions": {
                            "type": "string",
                            "description": "WHERE子句的具体查询条件",
                        },
                        "select_fields": {
                            "type": "string",
                            "description": "要选择的字段，用逗号分隔",
                        }
                    },
                    "required": ["table_info", "conditions", "select_fields"],
                },
            },
        }
    ]


    try:
        response = ollama_query(messages, tools)
        # response = gpt_4o_mini_query(messages, tools)
        print(response)
        assistant_message = response.choices[0].message
        if assistant_message.tool_calls:
            for tool_call in assistant_message.tool_calls:
                if tool_call.function.name == "generate_sql":
                    function_args = json.loads(tool_call.function.arguments)
                    sql_query = generate_sql(
                        function_args["table_info"],
                        function_args["conditions"],
                        function_args["select_fields"]
                    )
                    results = execute_sql(sql_query)
                    formatted_results = format_results(results, function_args["select_fields"].split(", ") if function_args["select_fields"] != "*" else None)
                    return f"生成的SQL查询: {sql_query}\n\n结果:\n{formatted_results}"

        return "无法生成SQL查询。请尝试重新表述您的问题。"

    except Exception as e:
        return f"发生错误: {str(e)}"


result = run_text2sql_conversation("Query Ivy age")
print(result)

# 主程序
# if __name__ == "__main__":
#     print("欢迎使用Text2SQL系统！")
#     print("您可以用自然语言询问有关用户表的问题。")
#     print("输入'quit'退出程序。")

#     while True:
#         user_input = input("\n请输入您的查询 (或 'quit' 退出): ")
#         if user_input.lower() == 'quit':
#             print("谢谢使用，再见！")
#             break

#         result = run_text2sql_conversation(user_input)
#         print("\n" + "="*50)
#         print(result)
#         print("="*50)

ChatCompletion(id='chatcmpl-56', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_g1ro8h87', function=Function(arguments='{"conditions":"name = \'Ivy\'","select_fields":"age","table_info":"users(id INTEGER, name TEXT, age INTEGER, email TEXT, registration_date DATE, last_login DATETIME)"}', name='generate_sql'), type='function')]))], created=1721617872, model='llama3-groq-tool-use:8b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=65, prompt_tokens=248, total_tokens=313))
生成的SQL查询: SELECT age FROM users WHERE name = 'Ivy'

结果:
30
28
34
65
49
20
